In [1]:
import json
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split

# Сформировать множество из разных файлов

In [2]:
df = pd.read_csv("../../Data/Raw/merged_with_labels_11_patterns.csv", sep=";")
print(len(df[df['x'].isna()]), "nan texts")
df = df[~df['x'].isna()]

df_3 = pd.read_csv("../../Data/Raw/12052022_one_cmd.csv", sep=";")

df_3.drop("delayed", axis=1, inplace=True)

0 nan texts


In [3]:
df = pd.concat([df_3, df[df["type"].isin(["students", "crowdsource"])]])

In [4]:
df.drop_duplicates(inplace=True)

# предобработка

In [5]:
# в случае изменения функции, изменить её и в обработке длинных команд в Analyze_complex_commands.ipynb
#punct_characters = re.escape(".,!?:;-#$*()[]'\"/|^%~`_+-")
import re
def removePunctETC(value):
    if type(value) is float:
        return value
    value = value.lower()
    value = re.sub("[^\w\s°]", "", value)
    return value
df["x"] = df["x"].map(removePunctETC)
df.drop_duplicates(inplace=True)

# сплит

In [6]:
# сплит сгенерированной части на трейн и валид 
print(df["type"].unique())
traindf = df[~df["type"].isin(['students', 'crowdsource'])]

validIndex = set()
for c in traindf.columns:
    if c in ["x", "y_name", "type"]:
        continue
    train_x, valid_x = train_test_split(traindf.index, test_size=0.03, random_state=69, shuffle=True, stratify=traindf[c])
    validIndex |= set(valid_x)
print("valid percent", len(validIndex) / df.shape[0])
subsets = ["valid" if i in validIndex else "train" for i in range(df.shape[0])]
df["subset"] = subsets
df.loc[df["type"].isin(['students', 'crowdsource']),"subset"] = "test"

['generator' 'students' 'crowdsource']
valid percent 0.22322980782784382


In [7]:
for c in df.columns:
    if c in ["x", "y_name", "subset", "type"]:
        continue
    trains_counts = df.loc[df["subset"]=="train", c].value_counts().to_dict()
    valid_counts = df.loc[df["subset"]=="valid", c].value_counts().to_dict()
    test_counts = df.loc[df["subset"]=="test", c].value_counts().to_dict()
    print(c)
    for k in trains_counts:
        if k in test_counts:
            print(k, trains_counts[k], valid_counts[k], test_counts[k], valid_counts[k] / trains_counts[k], test_counts[k] / trains_counts[k])
        else:
            print(k, trains_counts[k], valid_counts[k], valid_counts[k] / trains_counts[k])
    print()

y
9 1725 254 0.1472463768115942
3 1647 340 0.20643594414086217
2 1543 456 0.29552819183408946
11 1458 457 0.31344307270233196
7 1431 569 130 0.3976240391334731 0.09084556254367575
4 1385 409 0.29530685920577615
6 1366 484 0.35431918008784774
8 1331 669 995 0.5026296018031555 0.7475582268970699
10 1309 501 0.3827349121466769
5 1080 404 53 0.37407407407407406 0.049074074074074076
1 924 192 85 0.2077922077922078 0.09199134199134198
0 28 10 48 0.35714285714285715 1.7142857142857142

action
5 3032 749 0.2470316622691293
6 2783 1044 1178 0.37513474667624863 0.4232842256557672
8 1799 686 0.38132295719844356
9 1788 693 0.3875838926174497
0 1738 256 30 0.1472957422324511 0.01726121979286536
7 1605 661 0.4118380062305296
4 1543 456 0.29552819183408946
2 734 160 52 0.21798365122615804 0.07084468664850137
3 190 32 33 0.16842105263157894 0.1736842105263158
1 15 8 18 0.5333333333333333 1.2

direction
0 9728 3348 1226 0.3441611842105263 0.1260279605263158
3 1398 348 9 0.24892703862660945 0.0064377682

In [8]:
df.to_csv("../../Data/Interim/12052022_one_cmd_split.csv", index=False)
#df = pd.read_csv("../../Data/Interim/merged_with_labels_11_patterns_split.csv")
#df.sample(2000).to_csv("../../Data/Interim/short_split.csv", index=False)

# folds

In [9]:
from sklearn.model_selection import StratifiedKFold
df = pd.read_csv("../../Data/Interim/12052022_one_cmd_split.csv")
df_old_folds = pd.read_csv("../../Data/Interim/merged_with_labels_11_patterns_split.csv")
df["fold"]=-1
df[df["subset"]=="test"]["y"].value_counts()

8    995
7    130
1     85
5     53
0     48
Name: y, dtype: int64

In [10]:
skf = StratifiedKFold(n_splits=5)

for fold_i, (train_index, test_index) in enumerate(skf.split(df[df["subset"]=="test"].index, df[df["subset"]=="test"]["y"])):
    df.loc[df[df["subset"]=="test"].index[test_index], "fold"] = fold_i
    print(len(train_index), len(test_index))
for i in range(5):
    print("fold#", i)
    print(df[df["fold"]==i]["y"].value_counts())

1048 263
1049 262
1049 262
1049 262
1049 262
fold# 0
8    199
7     26
1     17
5     11
0     10
Name: y, dtype: int64
fold# 1
8    199
7     26
1     17
0     10
5     10
Name: y, dtype: int64
fold# 2
8    199
7     26
1     17
0     10
5     10
Name: y, dtype: int64
fold# 3
8    199
7     26
1     17
5     11
0      9
Name: y, dtype: int64
fold# 4
8    199
7     26
1     17
5     11
0      9
Name: y, dtype: int64


In [11]:
df.to_csv("../../Data/Interim/12052022_one_cmd_split.csv", index=False)